# Clasificación de iris con red neuronal simple

In [6]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
 
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=42)
 
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [7]:
import torch
import torch.nn as nn

class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)  # Input layer
        self.relu = nn.ReLU()                          # Activation function
        self.fc2 = nn.Linear(hidden_size, output_size) # Output layer
         
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x    

In [8]:
torch.manual_seed(42) # Fijamos la semilla para asegurar reproducibilidad

input_size = iris.data.shape[1] # Capa de entrada: número de características
hidden_size = 10 # Tamaño de la capa oculta
output_size = len(iris.target_names) # Capa de salida: número de clases

model = SimpleNN(input_size, hidden_size, output_size) # Instanciamos el modelo

criterion = nn.CrossEntropyLoss() # Función de pérdida
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr=0.01) # Optimizador

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train)

num_epochs = 100
for epoch in range(num_epochs):
    outputs = model(X_train_tensor) # Forward pass
    loss = criterion(outputs, y_train_tensor) # Calcular la pérdida
    
    optimizer.zero_grad() # Inicializar los gradientes a cero
    loss.backward() # Backward pass
    optimizer.step() # Actualizar los parámetros
    
    if (epoch+1) % 10 == 0: # Cada 10 epochs muestra la pérdida
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}') 

Epoch [10/100], Loss: 0.7783
Epoch [20/100], Loss: 0.5399
Epoch [30/100], Loss: 0.3921
Epoch [40/100], Loss: 0.2934
Epoch [50/100], Loss: 0.2166
Epoch [60/100], Loss: 0.1639
Epoch [70/100], Loss: 0.1284
Epoch [80/100], Loss: 0.1050
Epoch [90/100], Loss: 0.0902
Epoch [100/100], Loss: 0.0800


In [9]:
with torch.no_grad():
	X_test_tensor = torch.FloatTensor(X_test)
	y_test_tensor = torch.LongTensor(y_test)
	outputs = model(X_test_tensor)
	_, predicted = torch.max(outputs, 1)
	accuracy = (predicted == y_test_tensor).sum().item() / len(y_test_tensor)
	print(f'Accuracy on the test set: {accuracy:.2f}')

Accuracy on the test set: 0.97
